In [ ]:
import requests, zipfile, io, re, pandas

api = 'https://api.earthref.org/v1/MagIC/{}'
api = 'http://localhost:3200/v1/MagIC/{}'

### Given a MagIC contribution ID, download the MagIC contribution text file if it is public

In [ ]:
# Try to download the contribution text for an invalid ID
response = requests.get(api.format('download'), params={'id': 'a'})
print(response.request.method, response.request.url)
print('Error:', response.json()['errors'][0]['message'], '\n')

# Try to download the contribution text for a could valid, but nonexistant, IDs
response = requests.get(api.format('download'), params={'id': [1000, 1001]})
print(response.request.method, response.request.url)
if (response.status_code == 204):
    print('Public contribution IDs 1000 and 1001 are not found in MagIC', '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Download the contribution text for a valid ID and print the first 200 characters
contribution_id = 16761
response = requests.get(api.format('download'), params={'id': contribution_id})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    contribution_text = io.TextIOWrapper(contribution_zip.open('{}/magic_contribution_{}.txt'.format(contribution_id, contribution_id))).read()
    print(contribution_text[0:200], '\n')
elif (response.status_code == 204):
    print('Public contribution ID = \'{}\' not found in MagIC'.format(contribution_id), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Get the ID for the latest public MagIC contribution
latest_contribution_id_response = requests.get(api.format('search/contributions?n_max_rows=1'))
print(latest_contribution_id_response.request.method, latest_contribution_id_response.request.url)
if (latest_contribution_id_response.status_code == 200):
    latest_contribution_id = latest_contribution_id_response.json()['results'][0]['id']
elif (latest_contribution_id_response.status_code == 204):
    print('Latest contribution ID not retrieved in MagIC', '\n')
else:
    print('Error:', latest_contribution_id_response.json()['errors'][0]['message'], '\n')

# Get the contribution text for the ID and print the first 200 characters
latest_contribution_response = requests.get(api.format('download'), params={'id': latest_contribution_id})
print(latest_contribution_response.request.method, latest_contribution_response.request.url)
if (latest_contribution_response.status_code == 200):
    latest_contribution_zip = zipfile.ZipFile(io.BytesIO(latest_contribution_response.content))
    latest_contribution_text = io.TextIOWrapper(latest_contribution_zip.open('{}/magic_contribution_{}.txt'.format(latest_contribution_id, latest_contribution_id))).read()
    print(latest_contribution_text[0:200], '\n')
elif (latest_contribution_response.status_code == 204):
    print('Public contribution ID = \'{}\' not found in MagIC'.format(latest_contribution_id), '\n')
else:
    print('Error:', latest_contribution_response.json()['err'][0]['message'], '\n')

### Given a DOI, download the MagIC contribution text file for the latest public version(s)

In [ ]:
# Download and extract the latest contribution file to a local downloads folder
reference_doi = '10.1016/J.PEPI.2017.05.009'
response = requests.get(api.format('download'), params={'doi': reference_doi, 'n_max_contributions': 1})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    for filename in contribution_zip.namelist():
        if (re.match(r'^\d+\/magic_contribution_\d+\.txt', filename)):
            contribution_zip.extract(filename, 'downloads')
            print(filename, 'extracted to the downloads folder', '\n')
elif (response.status_code == 204):
    print('Public contribution with a reference DOI = \'{}\' not found in MagIC'.format(reference_doi), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Download and extract the latest contribution file to a magic_contribution.txt file
reference_doi = '10.1016/J.PEPI.2017.05.009'
response = requests.get(api.format('download'), params={'doi': reference_doi, 'n_max_contributions': 1})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    for filename in contribution_zip.namelist():
        if (re.match(r'^\d+\/magic_contribution_\d+\.txt', filename)):
            contribution_text = io.TextIOWrapper(contribution_zip.open(filename)).read()
            with open('downloads/magic_contribution.txt', 'wt') as fh:
                fh.write(contribution_text)
            print(filename, 'extracted to downloads/magic_contribution.txt', '\n')
elif (response.status_code == 204):
    print('Public contribution with a reference DOI = \'{}\' not found in MagIC'.format(reference_doi), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

# Get the contribution text for each matching contribution and print the first 200 characters
reference_doi = '10.1016/J.EPSL.2014.12.034'
response = requests.get(api.format('download'), params={'doi': reference_doi, 'n_max_contributions': 2})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    for filename in contribution_zip.namelist():
        if (re.match(r'^\d+\/magic_contribution_\d+\.txt', filename)):
            contribution_text = io.TextIOWrapper(contribution_zip.open(filename)).read()
            print(contribution_text[0:200], '\n')
elif (response.status_code == 204):
    print('Public contributions with a reference DOI = \'{}\' not found in MagIC'.format(reference_doi), '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

### Get the 50 latest public sites in MagIC that mention basalt or sandstone

In [ ]:
# Get the matching sites rows into a Pandas dataframe
response = requests.get(api.format('search/sites'), params={'query': 'basalt OR sandstone', 'n_max_rows': 50})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    sites = response.json()['results']
elif (response.status_code == 204):
    print('No sites in MagIC match the search parameters.', '\n')
else:
    print('Error:', response.json()['errors'][0]['message'], '\n')

sites_df = pandas.DataFrame(sites)
sites_df.head(50)